# Pass the Hash Activity 2
Detects the attack technique pass the hash which is used to move laterally inside the network

## Rule Content
```
- title: Pass the Hash Activity 2
  id: 8eef149c-bd26-49f2-9e5a-9b00e3af499b
  status: production
  description: Detects the attack technique pass the hash which is used to move laterally
    inside the network
  references:
  - https://github.com/iadgov/Event-Forwarding-Guidance/tree/master/Events
  - https://blog.binarydefense.com/reliably-detecting-pass-the-hash-through-event-log-analysis
  - https://blog.stealthbits.com/how-to-detect-pass-the-hash-attacks/
  author: Dave Kennedy, Jeff Warren (method) / David Vassallo (rule)
  tags:
  - attack.lateral_movement
  - attack.t1075
  logsource:
    product: windows
    service: security
    definition: The successful use of PtH for lateral movement between workstations
      would trigger event ID 4624
    category: null
  detection:
    selection:
    - EventID: 4624
      SubjectUserSid: S-1-0-0
      LogonType: '3'
      LogonProcessName: NtLmSsp
      KeyLength: '0'
    - EventID: 4624
      LogonType: '9'
      LogonProcessName: seclogo
    filter:
      AccountName: ANONYMOUS LOGON
    condition: selection and not filter
  falsepositives:
  - Administrator activity
  - Penetration tests
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"4624" AND ((SubjectUserSid:"S\-1\-0\-0" AND logon_type:"3" AND logon_process_name:"NtLmSsp" AND KeyLength:"0") OR (logon_type:"9" AND logon_process_name:"seclogo"))) AND (NOT (user_name:"ANONYMOUS\ LOGON")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()